# Analyzing Rorschach Artblocks

This notebook analyzes the Rorschach Art Blocks collection with a focus on their traits and the prices they can realize. [Visit the repository](https://github.com/ymyke) for more notebooks on other projects, to use the code, or to contribute.

Important: This notebook uses [Plotly](https://plotly.com/python/) mostly to visualize data. Plotly charts won't be rendered in Githubs notebook viewer. So please [view this notebook on nbviewer](https://nbviewer.org/github/ymyke/niftylytics/blob/main/rorschachs.ipynb) (or on your local installation after cloning the repo). Note that nbviewer caches notebooks and sometimes takes a long time to pick up a new version.

On the upside, all the Plotly charts are interactive. You can zoom, hover, pan, etc. Zooming is especially important due to the outliers in some the data so you can zoom in on the interesting parts.

In [1]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Get the Rorschachs

Note that this takes a couple of minutes. Also note the timestamp below,
that is when the information was last retrieved.

In [2]:
import datetime
from opensea import ARTBLOCK_CONTRACT, retrieve_assets

RORSCHACH_IDS = list(range(188000000, 188001024))
assets = retrieve_assets(token_ids=RORSCHACH_IDS, contract=ARTBLOCK_CONTRACT)
print(datetime.datetime.now())

.................................................... -- All 1024 assets retrieved.
2021-11-01 21:46:29.506930


## Build a dataframe

In [3]:
import numpy as np
import pandas as pd
from helpers import turn_assets_into_df
df, traits = turn_assets_into_df(assets)
hoverdata = [ "Name", *traits, "Probscore"]

In [4]:
assert set(df.LPsymbol.unique()) == set(["ETH", "WETH", np.nan]), """
This sheet does not do currency conversion at the moment and therefore 
assumes all prices are in (W)ETH. But there are more symbols in the input 
data which would lead to apples being compared to ranges below. Aborting.
"""

## How many pieces have a last price on OpenSea?

In [5]:
print(f"{df[~df.Lastprice.isna()].shape[0] / df.shape[0]:.0%}")

13%


## How are prices evolving over time?

Note that this only takes into account the last sale of each piece.

In [6]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 8)
pd.plotting.register_matplotlib_converters()

fig = px.scatter(df, x="LPdate", y="Lastprice", hover_data=hoverdata)
fig.show()

## Price levels today

In [7]:
fig = px.histogram(df, x="Lastprice", marginal="box", hover_data=hoverdata)
fig.show()

## How frequent are the different traits?

In [8]:
for trait in traits:
    display(
        pd.DataFrame(df[trait].value_counts(normalize=False, sort=True, ascending=True)).transpose()
    )


,colored,bw
headcolor,206,818


,eater,lenser,devo,plus,doubler,block
headshape,149,157,173,178,179,188


,3,4,8,1,5,7,6,2
swatch,100,115,115,130,135,139,139,151


## How much do people pay for the different traits?

In [9]:
for trait in traits:

    fig = go.Figure()
    for traitvariant in (
        df.groupby(trait).median().sort_values("Lastprice", ascending=False).index
    ):
        fig.add_trace(
            go.Box(
                y=df[df[trait] == traitvariant].Lastprice.values,
                name=traitvariant,
                boxpoints="all",
                jitter=0.2,
                whiskerwidth=0.2,
                marker_size=2,
                line_width=1,
            )
        )
    fig.update_layout(title=trait)
    fig.show()


## Probscore as a simple unified metric to predict prices

The Probscore metric is simply the product of the probabilities of the different traits of a piece. There is currently not much value in that metric. While only the lowest probabilities are able to command the very high outlier prices, overall correlation between Probscore and Lastprice remains low:

In [19]:
fig = px.scatter(df, x="Probscore", y="Lastprice", hover_data=hoverdata, color=df["headcolor"])
fig.show()
df[["Probscore", "Lastprice"]].corr()

,Probscore,Lastprice
Probscore,1.000000,-0.250066
Lastprice,-0.250066,1.000000
